In [1]:
# Ignore  the warnings
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import time
import random

from sklearn.utils import shuffle

from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import cross_val_score

from sklearn.svm import LinearSVC,SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

from sklearn.decomposition import PCA

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import cohen_kappa_score

#from sklearn.datasets import fetch_olivetti_faces
#of= fetch_olivetti_faces()

In [2]:
data=pd.read_table('faces.txt',' ',header=None)

In [3]:
scaler=MinMaxScaler()
scaler.fit(data)
data=scaler.transform(data)
data=pd.DataFrame(data)
Y = np.array([i for i in range(1,41,1)])
Y = np.repeat(Y, [10]*40, axis=0)
data.insert(0,'Y',Y)

In [4]:
def generate_mask(tem,n_samples,n_categories):
    n_samples_per_class=n_samples//n_categories#每类样本数    
    return list(pd.Series(range(n_samples_per_class)).isin(tmp))*n_categories

#系统抽样
def system_sampling(n_samples,n_categories):
    '''
    输入：样本数量，类型数
    返回：训练集、开发集、测试集的索引

    '''
    n_samples_per_class=n_samples//n_categories#每类样本数
    
    train_tem=random.sample(range(n_samples_per_class),int(0.6*n_samples_per_class))
    
    dev_test=list(set(range(n_samples_per_class))-set(train_tem))
    
    dev_tem=random.sample(dev_test,int(0.2*n_samples_per_class))
    
    test_tem=list(set(dev_test)-set(dev_tem))
    
    train_index=generate_mask(train_tem,n_samples,n_categories)
    
    test_index=generate_mask(test_tem,n_samples,n_categories)
    
    dev_index=generate_mask(dev_tem,n_samples,n_categories)
    
    return train_index,test_index,dev_index

In [5]:
train_index,validation_index,test_index=system_sampling(400,40)

In [6]:
data_test=data.loc[test_index]
data_train=data.loc[train_index]
data_validation=data.loc[validation_index]

In [7]:
x_train=np.array(data_train[data_train.columns.difference(['Y'])])
y_train=np.array(data_train['Y'])

x_val=np.array(data_validation[data_validation.columns.difference(['Y'])])
y_val=np.array(data_validation['Y'])

x_test=np.array(data_test[data_test.columns.difference(['Y'])])
y_test=np.array(data_test['Y'])

In [8]:
models=[GaussianNB(),SGDClassifier(),KNeighborsClassifier()
        ,LinearSVC(),LogisticRegression(),SVC(kernel='rbf')
        ,DecisionTreeClassifier(),AdaBoostClassifier(),GradientBoostingClassifier()
        ,RandomForestClassifier(),BaggingClassifier(),ExtraTreesClassifier()
        ,XGBClassifier()]

model_names=['GaussianNB','SGDClassifier','KNearestNeighbors'
             ,'LinearSVM','LogisticRegression','rbfSVM'
             ,'DecisionTree','AdaBoostClassifier','GradientBoostingClassifier'
             ,'RandomForestClassifier','BaggingClassifier','ExtraTreesClassifier'
             ,'xgboost'
]

accuracy=[]
clf=[]
t=[]
i=0
for m in range(len(models)):
    clf.append(models[m])
    time_start = time.time()
    clf[i].fit(x_train,y_train)
    pred=clf[i].predict(x_val)
    accuracy.append(accuracy_score(pred,y_val))
    time_end = time.time()
    t.append(time_end-time_start)
    i=i+1
result_frame1=pd.DataFrame({'Model':model_names,'Validation set accuracy':accuracy,'Time':t})

In [9]:
result_frame1

,Model,Validation set accuracy,Time
0,GaussianNB,0.8125,0.201461
1,SGDClassifier,0.2875,0.277258
2,KNearestNeighbors,0.7750,0.173537
3,LinearSVM,0.9375,10.866472
4,LogisticRegression,0.9375,5.204164
5,rbfSVM,0.7500,0.801859
6,DecisionTree,0.5375,1.718409
7,AdaBoostClassifier,0.1500,4.264610
8,GradientBoostingClassifier,0.5000,46.531203
9,RandomForestClassifier,0.6750,0.153590


调参

1.SVM

In [11]:
SVM_time_start1 = time.time()
kernel=['linear','poly','rbf','sigmoid']
for k in kernel:
    clf=SVC(kernel=k)
    clf.fit(x_train,y_train)
    pred=clf.predict(x_val)
    print('Kernel',k,'准确率：',accuracy_score(pred,y_val))
SVM_time_end1 = time.time()

Kernel linear 准确率： 0.9375
Kernel poly 准确率： 0.7
Kernel rbf 准确率： 0.75
Kernel sigmoid 准确率： 0.7375


In [12]:
SVM_time_end1-SVM_time_start1

2.998931407928467

In [13]:
SVM_time_start2 = time.time()
C=[0.01,0.1,1,10,100,1000]
gamma=[0.01,0.1,1,10,100,1000]
for c in C:
    for g in gamma:
        clf=SVC(C=c,kernel='linear',gamma=g)
        clf.fit(x_train,y_train)
        pred=clf.predict(x_val)
        print('参数：','C:',c,'gamma:',g,'准确率：',accuracy_score(pred,y_val))
SVM_time_end2 = time.time()

参数： C: 0.01 gamma: 0.01 准确率： 0.925
参数： C: 0.01 gamma: 0.1 准确率： 0.925
参数： C: 0.01 gamma: 1 准确率： 0.925
参数： C: 0.01 gamma: 10 准确率： 0.925
参数： C: 0.01 gamma: 100 准确率： 0.925
参数： C: 0.01 gamma: 1000 准确率： 0.925
参数： C: 0.1 gamma: 0.01 准确率： 0.9375
参数： C: 0.1 gamma: 0.1 准确率： 0.9375
参数： C: 0.1 gamma: 1 准确率： 0.9375
参数： C: 0.1 gamma: 10 准确率： 0.9375
参数： C: 0.1 gamma: 100 准确率： 0.9375
参数： C: 0.1 gamma: 1000 准确率： 0.9375
参数： C: 1 gamma: 0.01 准确率： 0.9375
参数： C: 1 gamma: 0.1 准确率： 0.9375
参数： C: 1 gamma: 1 准确率： 0.9375
参数： C: 1 gamma: 10 准确率： 0.9375
参数： C: 1 gamma: 100 准确率： 0.9375
参数： C: 1 gamma: 1000 准确率： 0.9375
参数： C: 10 gamma: 0.01 准确率： 0.9375
参数： C: 10 gamma: 0.1 准确率： 0.9375
参数： C: 10 gamma: 1 准确率： 0.9375
参数： C: 10 gamma: 10 准确率： 0.9375
参数： C: 10 gamma: 100 准确率： 0.9375
参数： C: 10 gamma: 1000 准确率： 0.9375
参数： C: 100 gamma: 0.01 准确率： 0.9375
参数： C: 100 gamma: 0.1 准确率： 0.9375
参数： C: 100 gamma: 1 准确率： 0.9375
参数： C: 100 gamma: 10 准确率： 0.9375
参数： C: 100 gamma: 100 准确率： 0.9375
参数： C: 100 gamma: 1000 准确率： 0.9375
参数：

In [14]:
SVM_time_end2-SVM_time_start2

24.425994634628296

2.GaussianNB

In [15]:
NB_time_start = time.time()
var_smoothing=[10**-i for i in range(10)]
for v in var_smoothing:
        clf=GaussianNB(var_smoothing=v)
        clf.fit(x_train,y_train)
        pred=clf.predict(x_val)
        print('参数：','var_smoothing:',v,'准确率：',accuracy_score(pred,y_val))
NB_time_end = time.time()

参数： var_smoothing: 1 准确率： 0.7375
参数： var_smoothing: 0.1 准确率： 0.8875
参数： var_smoothing: 0.01 准确率： 0.8375
参数： var_smoothing: 0.001 准确率： 0.8125
参数： var_smoothing: 0.0001 准确率： 0.8125
参数： var_smoothing: 1e-05 准确率： 0.8125
参数： var_smoothing: 1e-06 准确率： 0.8125
参数： var_smoothing: 1e-07 准确率： 0.8125
参数： var_smoothing: 1e-08 准确率： 0.8125
参数： var_smoothing: 1e-09 准确率： 0.8125


In [16]:
NB_time_end-NB_time_start

2.565253734588623

3.KNearestNeighbors

In [17]:
KNN_time_start = time.time()
n_neighbors=[i for i in range(1,11)]
for n in n_neighbors:
        clf=KNeighborsClassifier(n_neighbors=n)
        clf.fit(x_train,y_train)
        pred=clf.predict(x_val)
        print('参数：','n_neighbors=:',n,'准确率：',accuracy_score(pred,y_val))
KNN_time_end = time.time()

参数： n_neighbors=: 1 准确率： 0.9125
参数： n_neighbors=: 2 准确率： 0.825
参数： n_neighbors=: 3 准确率： 0.8125
参数： n_neighbors=: 4 准确率： 0.7875
参数： n_neighbors=: 5 准确率： 0.775
参数： n_neighbors=: 6 准确率： 0.7375
参数： n_neighbors=: 7 准确率： 0.7125
参数： n_neighbors=: 8 准确率： 0.675
参数： n_neighbors=: 9 准确率： 0.65
参数： n_neighbors=: 10 准确率： 0.6125


In [19]:
KNN_time_end-KNN_time_start

1.785017490386963

4.LogisticRegression

In [18]:
LR_time_start = time.time()
LR_accuracy=[]
penalty=['l1','l2']
C=[0.001, 0.01, 0.1, 1, 10, 100, 1000]
for p in penalty:
    for c in C:
        clf=LogisticRegression(penalty=p,C=c)
        clf.fit(x_train,y_train)
        pred=clf.predict(x_val)
        print('参数：','penalty:',p,'C:',c,'准确率：',accuracy_score(pred,y_val))
        LR_accuracy.append(accuracy_score(pred,y_val))
LR_time_end = time.time()

参数： penalty: l1 C: 0.001 准确率： 0.025
参数： penalty: l1 C: 0.01 准确率： 0.025
参数： penalty: l1 C: 0.1 准确率： 0.1125
参数： penalty: l1 C: 1 准确率： 0.725
参数： penalty: l1 C: 10 准确率： 0.8875
参数： penalty: l1 C: 100 准确率： 0.9125
参数： penalty: l1 C: 1000 准确率： 0.925
参数： penalty: l2 C: 0.001 准确率： 0.475
参数： penalty: l2 C: 0.01 准确率： 0.875
参数： penalty: l2 C: 0.1 准确率： 0.9375
参数： penalty: l2 C: 1 准确率： 0.9375
参数： penalty: l2 C: 10 准确率： 0.925
参数： penalty: l2 C: 100 准确率： 0.925
参数： penalty: l2 C: 1000 准确率： 0.925


In [21]:
LR_time_end-LR_time_start

50.156182050704956

5.ExtraTreesClassifier

In [22]:
ET_time_start = time.time()
n_estimators=[100,200,300,400,500,600]
for n in n_estimators:
        clf=ExtraTreesClassifier(n_estimators=n)
        clf.fit(x_train,y_train)
        pred=clf.predict(x_val)
        print('参数：','n_estimators=',n,'准确率：',accuracy_score(pred,y_val))
ET_time_end = time.time()

参数： n_estimators= 100 准确率： 0.925
参数： n_estimators= 200 准确率： 0.925
参数： n_estimators= 300 准确率： 0.9375
参数： n_estimators= 400 准确率： 0.9375
参数： n_estimators= 500 准确率： 0.925
参数： n_estimators= 600 准确率： 0.9375


In [23]:
ET_time_end-ET_time_start

7.453470468521118

模型评估

In [24]:
def evaluation(clf):
    clf.fit(x_train,y_train)
    y_pre=clf.predict(x_test)
    print('confusion_matrix:','\n')
    print(confusion_matrix(y_test, y_pre),'\n')
    print('accuracy:',accuracy_score(y_test, y_pre),';','kappa',cohen_kappa_score(y_test, y_pre))

In [25]:
evaluation(SVC(C=1,kernel='linear',gamma=1))

confusion_matrix: 

[[2 0 0 ... 0 0 0]
 [0 2 0 ... 0 0 0]
 [0 0 2 ... 0 0 0]
 ...
 [0 0 0 ... 2 0 0]
 [0 0 0 ... 0 2 0]
 [0 0 0 ... 0 0 2]] 

accuracy: 0.9875 ; kappa 0.9871794871794872


In [26]:
evaluation(GaussianNB(var_smoothing=0.1))

confusion_matrix: 

[[1 0 0 ... 0 0 0]
 [0 2 0 ... 0 0 0]
 [0 0 2 ... 0 0 0]
 ...
 [0 0 0 ... 2 0 0]
 [0 0 0 ... 0 2 0]
 [0 0 0 ... 0 0 0]] 

accuracy: 0.925 ; kappa 0.9230769230769231


In [27]:
evaluation(KNeighborsClassifier(n_neighbors=1))

confusion_matrix: 

[[0 0 0 ... 1 0 0]
 [0 2 0 ... 0 0 0]
 [0 0 2 ... 0 0 0]
 ...
 [0 0 0 ... 2 0 0]
 [0 0 0 ... 0 2 0]
 [0 0 0 ... 0 0 1]] 

accuracy: 0.9375 ; kappa 0.9358974358974359


In [28]:
evaluation(LogisticRegression(penalty='l2',C=0.1))

confusion_matrix: 

[[2 0 0 ... 0 0 0]
 [0 2 0 ... 0 0 0]
 [0 0 2 ... 0 0 0]
 ...
 [0 0 0 ... 2 0 0]
 [0 0 0 ... 0 2 0]
 [0 0 0 ... 0 0 1]] 

accuracy: 0.975 ; kappa 0.9743589743589743


In [31]:
evaluation(ExtraTreesClassifier(n_estimators=300))

confusion_matrix: 

[[2 0 0 ... 0 0 0]
 [0 2 0 ... 0 0 0]
 [0 0 2 ... 0 0 0]
 ...
 [0 0 0 ... 2 0 0]
 [0 0 0 ... 0 2 0]
 [0 0 0 ... 0 0 1]] 

accuracy: 0.975 ; kappa 0.9743589743589743
